In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [3]:
input_checkpoint = "model/model.ckpt"
output_graph = "./model.pb"

INFO:tensorflow:Restoring parameters from model/model.ckpt
INFO:tensorflow:Froze 248 variables.
INFO:tensorflow:Converted 248 variables to const ops.


In [ ]:
output_node_names = "fky_input,fky_output,model_1/dense_layer_1/LogSoftmax,model_1/flatten_layer_3/Tanh"
saver = tf.train.import_meta_graph(
    input_checkpoint + '.meta', clear_devices=True)
graph = tf.get_default_graph()  # 获得默认的图
input_graph_def = graph.as_graph_def()  # 返回一个序列化的图代表当前的图
with tf.Session() as sess:
    saver.restore(sess, input_checkpoint)  # 恢复图并得到数据
    output_graph_def = graph_util.convert_variables_to_constants(  # 模型持久化，将变量值固定
        sess=sess,
        input_graph_def=input_graph_def,  # 等于:sess.graph_def
        output_node_names=output_node_names.split(",")
    )

    with tf.gfile.GFile(output_graph, "wb") as f:  # 保存模型
        f.write(output_graph_def.SerializeToString())  
        
    # print("%d ops in the final graph." %
    #       len(output_graph_def.node))  # 得到当前图有几个操作节点
    # for op in graph.get_operations():
    #     print(op.name, op.values())

In [1]:
class Agent():
    def __init__(self, pb_path):
        with tf.Graph().as_default():
            output_graph_def = tf.GraphDef()
            with open(pb_path, "rb") as f:
                output_graph_def.ParseFromString(f.read())
                tf.import_graph_def(output_graph_def, name="")
            self.sess = tf.Session()

            self.sess.run(tf.global_variables_initializer())
            self.action_fc = self.sess.graph.get_tensor_by_name(
                "model_1/dense_layer_1/LogSoftmax:0")
            self.evaluation_fc2 = self.sess.graph.get_tensor_by_name(
                "model_1/flatten_layer_3/Tanh:0")
            self._input = self.sess.graph.get_tensor_by_name("fky_input:0")

    def calculate(self, state_batch):
        with tf.Graph().as_default():
            out, b = self.sess.run([self.action_fc, self.evaluation_fc2], feed_dict={self._input: state_batch})
            return out, b

